## Сравнение различных стратегий в дилемме заключённого

##### Класс экперимент и определение стратегий

In [12]:
import numpy as np
from typing import Callable, Tuple, List, Dict
from collections import defaultdict

class Experiment:
    
    def __init__(self, strategy_a: Callable, strategy_b: Callable, num_rounds: int = 200):

        self.strategy_a = strategy_a
        self.strategy_b = strategy_b
        self.num_rounds = num_rounds
        
        self.payoff_matrix = {
            (0, 0): (3, 3),  
            (0, 1): (0, 5),  
            (1, 0): (5, 0),  
            (1, 1): (1, 1)   
        }
        
        self.history_a = []  
        self.history_b = []  
        self.scores_a = []   
        self.scores_b = []   
        
    def play_round(self, round_num: int) -> Tuple[int, int]:
        prev_moves_a = self.history_a[:round_num]
        prev_moves_b = self.history_b[:round_num]
        
        move_a = self.strategy_a(prev_moves_a, prev_moves_b, round_num)
        move_b = self.strategy_b(prev_moves_b, prev_moves_a, round_num)
        
        return move_a, move_b
    
    def run_experiment(self) -> Dict:
        self.history_a = []
        self.history_b = []
        self.scores_a = []
        self.scores_b = []
        
        for round_num in range(self.num_rounds):
            move_a, move_b = self.play_round(round_num)
            
            self.history_a.append(move_a)
            self.history_b.append(move_b)
            
            score_a, score_b = self.payoff_matrix[(move_a, move_b)]
            self.scores_a.append(score_a)
            self.scores_b.append(score_b)

        return self.analyze_results()
    
    def analyze_results(self) -> Dict:
        total_score_a = sum(self.scores_a)
        total_score_b = sum(self.scores_b)

        dominant_series_a = self.find_dominant_series(self.scores_a, self.scores_b)
        dominant_series_b = self.find_dominant_series(self.scores_b, self.scores_a)
        
        return {
            'total_scores': (total_score_a, total_score_b),
            'dominant_series': (dominant_series_a, dominant_series_b),
            'moves_a': self.history_a,
            'moves_b': self.history_b,
            'scores_a': self.scores_a,
            'scores_b': self.scores_b
        }
    
    def find_dominant_series(self, scores1: List[int], scores2: List[int]) -> int:
        max_series = 0
        current_series = 0
        
        for s1, s2 in zip(scores1, scores2):
            if s1 == 5 and s2 == 0:
                current_series += 1
                max_series = max(max_series, current_series)
            else:
                current_series = 0
        
        return max_series

def strategy_alex(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 1

def strategy_bob(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 0

def strategy_clara(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    return opp_prev[-1] if opp_prev else 0

def strategy_denis(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    return 1 - opp_prev[-1] if opp_prev else 0

def strategy_emma(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 1 if (round_num + 1) % 20 == 0 else 0

def strategy_frida(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if not opp_prev:
        return 0
    return 0 if all(move == 0 for move in opp_prev) else 1

def strategy_merciful_clara(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    
    if (round_num + 1) % 10 == 0:
        return 0

    return opp_prev[-1] if opp_prev else 0

def calculate_strategy_rankings():
    strategies = {
        'Alex': strategy_alex,
        'Bob': strategy_bob,
        'Clara': strategy_clara,
        'Denis': strategy_denis,
        'Emma': strategy_emma,
        'Frida': strategy_frida,
        'merciful_Clara': strategy_merciful_clara
    }
    
    total_scores = defaultdict(int)
    total_games = defaultdict(int)
    max_series = defaultdict(int)
    avg_scores = {}
    
    strategy_names = list(strategies.keys())

    for i, name1 in enumerate(strategy_names):
        for j, name2 in enumerate(strategy_names):
            experiment = Experiment(strategies[name1], strategies[name2])
            result = experiment.run_experiment()
            
            total_scores[name1] += result['total_scores'][0]
            total_scores[name2] += result['total_scores'][1]
            
            max_series[name1] = max(max_series[name1], result['dominant_series'][0])
            max_series[name2] = max(max_series[name2], result['dominant_series'][1])
            
            total_games[name1] += 1
            total_games[name2] += 1
    
    for strategy in strategy_names:
        avg_scores[strategy] = total_scores[strategy] / total_games[strategy]
    
    sorted_by_avg_score = sorted(avg_scores.items(), key=lambda x: x[1], reverse=True)
    sorted_by_total_score = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
    sorted_by_max_series = sorted(max_series.items(), key=lambda x: x[1], reverse=True)
    
    return {
        'by_avg_score': sorted_by_avg_score,
        'by_total_score': sorted_by_total_score,
        'by_max_series': sorted_by_max_series,
        'total_scores': dict(total_scores),
        'avg_scores': avg_scores,
        'max_series': dict(max_series)
    }

def compare_all_strategies():
    strategies = {
        'Alex': strategy_alex,
        'Bob': strategy_bob,
        'Clara': strategy_clara,
        'Denis': strategy_denis,
        'Emma': strategy_emma,
        'Frida': strategy_frida,
        'merciful_Clara': strategy_merciful_clara
    }
    
    results = {}
    strategy_names = list(strategies.keys())
    
    for i, name1 in enumerate(strategy_names):
        for j, name2 in enumerate(strategy_names):
            if i <= j:
                experiment = Experiment(
                    strategies[name1], 
                    strategies[name2]
                )
                result = experiment.run_experiment()
                
                key = f"{name1} vs {name2}"
                results[key] = result
    
    return results, strategies



##### Сравнение стратегий

In [13]:
rankings = calculate_strategy_rankings()

print("=" * 60)
print("РЕЙТИНГ СТРАТЕГИЙ ПО СРЕДНЕМУ КОЛИЧЕСТВУ ОЧКОВ ЗА ИГРУ")
print("=" * 60)
for i, (strategy, score) in enumerate(rankings['by_avg_score'], 1):
    print(f"{i:2}. {strategy:15} {score:6.2f} очков")

print("\n" + "=" * 50)
print("РЕЙТИНГ СТРАТЕГИЙ ПО ОБЩЕМУ КОЛИЧЕСТВУ ОЧКОВ")
print("=" * 50)
for i, (strategy, score) in enumerate(rankings['by_total_score'], 1):
    print(f"{i:2}. {strategy:15} {score:6} очков")

print("\n" + "=" * 50)
print("РЕЙТИНГ СТРАТЕГИЙ ПО МАКСИМАЛЬНОЙ СЕРИИ ДОМИНИРОВАНИЯ")
print("=" * 50)
for i, (strategy, series) in enumerate(rankings['by_max_series'], 1):
    print(f"{i:2}. {strategy:15} {series:3} раундов")

РЕЙТИНГ СТРАТЕГИЙ ПО СРЕДНЕМУ КОЛИЧЕСТВУ ОЧКОВ ЗА ИГРУ
 1. Frida           644.14 очков
 2. Alex            550.29 очков
 3. Clara           519.57 очков
 4. merciful_Clara  510.00 очков
 5. Denis           470.71 очков
 6. Bob             424.71 очков
 7. Emma            358.14 очков

РЕЙТИНГ СТРАТЕГИЙ ПО ОБЩЕМУ КОЛИЧЕСТВУ ОЧКОВ
 1. Frida             9018 очков
 2. Alex              7704 очков
 3. Clara             7274 очков
 4. merciful_Clara    7140 очков
 5. Denis             6590 очков
 6. Bob               5946 очков
 7. Emma              5014 очков

РЕЙТИНГ СТРАТЕГИЙ ПО МАКСИМАЛЬНОЙ СЕРИИ ДОМИНИРОВАНИЯ
 1. Alex            200 раундов
 2. Denis           199 раундов
 3. Frida           197 раундов
 4. Clara             1 раундов
 5. Emma              1 раундов
 6. merciful_Clara    1 раундов
 7. Bob               0 раундов


##### Статистика каждой стратегии

In [14]:
print("\n" + "=" * 60)
print("ВСЕ СТРАТЕГИИ")
print("=" * 60)
for strategy in rankings['total_scores'].keys():
    print(f"{strategy:15} Всего очков: {rankings['total_scores'][strategy]:5}, "
          f"Среднее: {rankings['avg_scores'][strategy]:5.2f}, "
          f"Макс. серия: {rankings['max_series'][strategy]:2}")


ВСЕ СТРАТЕГИИ
Alex            Всего очков:  7704, Среднее: 550.29, Макс. серия: 200
Bob             Всего очков:  5946, Среднее: 424.71, Макс. серия:  0
Clara           Всего очков:  7274, Среднее: 519.57, Макс. серия:  1
Denis           Всего очков:  6590, Среднее: 470.71, Макс. серия: 199
Emma            Всего очков:  5014, Среднее: 358.14, Макс. серия:  1
Frida           Всего очков:  9018, Среднее: 644.14, Макс. серия: 197
merciful_Clara  Всего очков:  7140, Среднее: 510.00, Макс. серия:  1


##### Стратегия1 vs стратегия2

In [15]:
all_results, strategies = compare_all_strategies()
    
print("\nТАБЛИЦА ОБЩИХ ОЧКОВ:")
header = "Стратегия1 vs Стратегия2            | Очки1 | Очки2"
print(header)
print("-" * len(header))
    
for match, result in all_results.items():
    scores = result['total_scores']
    print(f"{match:35} | {scores[0]:5} | {scores[1]:5}")
    
print("\nТАБЛИЦА ДОМИНИРУЮЩИХ СЕРИЙ:")
header = "Стратегия1 vs Стратегия2            | Серия1 | Серия2"
print(header)
print("-" * len(header))
    
for match, result in all_results.items():
    series = result['dominant_series']
    print(f"{match:35} | {series[0]:6} | {series[1]:6}")


ТАБЛИЦА ОБЩИХ ОЧКОВ:
Стратегия1 vs Стратегия2            | Очки1 | Очки2
---------------------------------------------------
Alex vs Alex                        |   200 |   200
Alex vs Bob                         |  1000 |     0
Alex vs Clara                       |   204 |   199
Alex vs Denis                       |  1000 |     0
Alex vs Emma                        |   960 |    10
Alex vs Frida                       |   204 |   199
Alex vs merciful_Clara              |   284 |   179
Bob vs Bob                          |   600 |   600
Bob vs Clara                        |   600 |   600
Bob vs Denis                        |     3 |   998
Bob vs Emma                         |   570 |   620
Bob vs Frida                        |   600 |   600
Bob vs merciful_Clara               |   600 |   600
Clara vs Clara                      |   600 |   600
Clara vs Denis                      |   450 |   450
Clara vs Emma                       |   588 |   593
Clara vs Frida                      |   60